## Forex Data Collection
#### This notebook fetches historical forex candle data from the OANDA API, processes it into a structured format, and saves it in pickle files as currency pairs


In [1]:
import requests
import defs
import pandas as pd

In [2]:
session = requests.Session()

In [ ]:
# Load available instruments from a pickle file
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
# List of currency codes to fetch data for
currencies = ['EUR', 'USD', 'GBP', 'JYP', 'CHF', 'NZD', 'CAD']

In [5]:
# Fetch Candlestick Data
def fetch_candles(pair_name, count, granularity):
    """
    Fetch historical candlestick data for a given currency pair.
    
    Parameters:
    - pair_name (str): Currency pair (e.g., "EUR_USD")
    - count (int): Number of candles to retrieve
    - granularity (str): Timeframe (e.g., "H1" for 1-hour candles)

    Returns:
    - status_code (int): HTTP response status code
    - json_response (dict): JSON response containing candle data
    """
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"  # Mid, Bid, and Ask prices
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
code, res = fetch_candles("EUR_USD", 10, "H1")
code

200

In [7]:
# Convert JSON to DataFrame
def get_candles_df(json_response):
    """
    Convert JSON response of candle data into a pandas DataFrame.
    
    Parameters:
    - json_response (dict): API response containing candle data
    
    Returns:
    - pandas.DataFrame: Dataframe with OHLC and volume information
    """
    prices = ['mid', 'bid', 'ask']  # Price types
    ohlc = ['o', 'h', 'l', 'c']  # Open, High, Low, Close components

    historical_data = []
    for candle in json_response.get('candles', []): 
        if not candle.get('complete', False):  # Skip incomplete candles
            continue 

        # Store time and volume
        new_dict = {
            'time': candle['time'], 
            'volume': candle['volume']
        }

        # Extract OHLC prices for each price type
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        
        historical_data.append(new_dict)

    return pd.DataFrame.from_dict(historical_data)

In [8]:
# Save data to file
def save_file(candles_df, pair, granularity):
    """
    Save the processed candle data to a pickle file.
    
    Parameters:
    - candles_df (pandas.DataFrame): Dataframe containing historical data
    - pair (str): Currency pair
    - granularity (str): Timeframe
    """
    candles_df.to_pickle(f"hist_data/{pair}_{granularity}.pkl")

In [9]:
# Fetch and store data
def create_data(pair, granularity):
    """
    Fetch, process, and store candlestick data for a given currency pair.

    Parameters:
    - pair (str): Currency pair
    - granularity (str): Timeframe (e.g., "H1")

    If the API request fails, an error message is printed.
    """
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return

    df = get_candles_df(json_data)  # Convert response to DataFrame
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [10]:
# Process All Currency Pairs
for p1 in currencies:
    for p2 in currencies:
        pair = f"{p1}_{p2}"  # Construct currency pair name
        if pair in ins_df.name.unique():  # Check if pair is available
            create_data(pair, "H1")  # Fetch and save hourly candle data

EUR_USD loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2024-06-12T10:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
USD_CHF loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
USD_CAD loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
GBP_USD loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
GBP_CHF loaded 3999 candles from 2024-06-12T11:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
GBP_NZD loaded 3999 candles from 2024-06-12T09:00:00.000000000Z to 2025-02-04T02:00:00.000000000Z
GBP_CAD loaded 3999 